## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точность на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [5]:
import pandas as pd
import gensim
import pymorphy2
from pymorphy2 import tokenizers

c:\Users\Elizaveta\Classifications_2021\venv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
df_train = pd.read_csv('data/news_train.txt',sep='\t', header = None)

In [7]:
def clear_words(text):
    result = []
    for token in text:
        if (token.isalpha() or token.isdigit()) and len(token) > 2:
            result.append(token.lower())
    return result


In [8]:
df_train['words'] = df_train[2].apply(lambda x: tokenizers.simple_word_tokenize(x))
df_train['clean_words'] = df_train['words'].apply(lambda x: clear_words(x))

In [9]:
wtw = gensim.models.Word2Vec(sentences=df_train.clean_words.values.tolist(), min_count=1)

print(wtw['вирджиния'])
wtw.wv.similar_by_word('спорт')

[ 0.07006904 -0.08183089  0.05295759 -0.12090743 -0.04559543 -0.06939719
  0.09590948 -0.08637927 -0.05225879 -0.19005145 -0.03449963 -0.19282378
 -0.15321656 -0.02717497  0.0497912   0.1326137  -0.02787533 -0.00689011
  0.24356756  0.03049645  0.08533127  0.17156357 -0.10753623 -0.22629741
  0.12937261  0.15628245  0.20868163 -0.0087557  -0.05961137  0.18353494
 -0.08789812  0.10142672 -0.10274352 -0.08628326 -0.15028152  0.03478947
  0.05092376  0.05828741 -0.0691881   0.0778143   0.03327446  0.17047513
  0.0789312  -0.11953027 -0.22334813 -0.0633193   0.00250805  0.05873582
 -0.04054797  0.01446224  0.11731263  0.3121611   0.07406072 -0.2355761
  0.09579952 -0.20485735  0.04951693  0.1021158  -0.05489077  0.01147523
  0.05040554  0.18772642 -0.04129612 -0.05467477  0.02815236 -0.06677635
 -0.07669879  0.00670338  0.00472577  0.00804861  0.01940037  0.15781611
  0.14274769  0.06260297  0.00940195  0.04723461 -0.00186741  0.16366167
  0.20602639  0.0995039  -0.14652112 -0.19202656  0.

c:\Users\Elizaveta\Classifications_2021\venv\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('советский', 0.9735555052757263),
 ('bild', 0.932914137840271),
 ('телеканал', 0.926334798336029),
 ('архангельска', 0.9195427894592285),
 ('виталием', 0.9191847443580627),
 ('радиостанция', 0.917327880859375),
 ('радио', 0.9161995649337769),
 ('русская', 0.9158331155776978),
 ('прямом', 0.9108662009239197),
 ('телеканалу', 0.9066583514213562)]

In [10]:
themes = list(set(df_train[0].values))
print("Все темы - лейблы", themes)

Все темы - лейблы ['culture', 'travel', 'style', 'media', 'life', 'sport', 'science', 'forces', 'business', 'economics']


In [11]:
def themes_dist(themes, df):
    res_list = dict()
    for i in themes:
        res_list[i] = 0
    list_thems = df_train[0].values
    for i in list_thems:
        res_list[i] += 1
    for i in themes:
        res_list[i] /= len(list_thems)
    return res_list

def words_dist(words, themes, df):
    res_list = dict()
    for i in themes:
        for j in words:
            res_list[(i, j)] = 0
    
    list_thems = df_train[0].values.tolist()
    list_words = df_train.clean_words.tolist()
    count = dict()
    for i in themes:
        count[i] = 0
    for i in range(len(list_words)):
        for j in list_words[i]:
            res_list[(list_thems[i], j)] += 1
            count[list_thems[i]] += 1
    for label, feat in res_list:
        res_list[label, feat] /= count[label]
    return res_list

words_dict = list(set(wtw.wv.vocab.keys()))
distr_1 = themes_dist(themes, df_train)
distr_2 = words_dist(words_dict, themes, df_train)

distr_2

{('culture', 'телефонии'): 0.0,
 ('culture', 'пленоптических'): 0.0,
 ('culture', 'полуфиналиста'): 0.0,
 ('culture', 'запрашивается'): 0.0,
 ('culture', 'кларксоном'): 0.0,
 ('culture', 'галиной'): 3.2024287219427213e-06,
 ('culture', 'взлетные'): 3.2024287219427213e-06,
 ('culture', 'выгоден'): 0.0,
 ('culture', 'скипетр'): 3.2024287219427213e-06,
 ('culture', 'hawk'): 0.0,
 ('culture', 'адвокатам'): 3.2024287219427213e-06,
 ('culture', 'помилована'): 0.0,
 ('culture', 'согласовав'): 3.2024287219427213e-06,
 ('culture', 'деметриус'): 0.0,
 ('culture', 'венецианском'): 2.561942977554177e-05,
 ('culture', 'инджирлик'): 0.0,
 ('culture', 'группу'): 0.000195348152038506,
 ('culture', 'десятицентовик'): 0.0,
 ('culture', 'мрачность'): 0.0,
 ('culture', 'кафельников'): 0.0,
 ('culture', 'взять'): 4.1631573385255374e-05,
 ('culture', 'нацбезопасности'): 0.0,
 ('culture', 'астронавтами'): 0.0,
 ('culture', 'веранда'): 0.0,
 ('culture', 'модуль'): 0.0,
 ('culture', 'яценюк'): 0.0,
 ('culture'

In [12]:
from math import log
def classif(sentence, classes, prob):
    return max(classes.keys(),              # calculate argmax(-log(C|O))
        key = lambda cl: -log(classes[cl]) + sum(-log(prob.get((cl,feat), 0) + 10**(-10), 10**(-10)) for feat in sentence))

In [13]:
reses_train = []
clean_words = df_train.clean_words.values.tolist()
for i in range(len(clean_words)):
    reses_train.append([classif(clean_words[i], distr_1, distr_2), df_train[0].values.tolist()[i]])
    
right_count = 0
for i in reses_train:
    if (i[0] == i[1]):
        right_count += 1
print("Наивный Байессовский классификатор на обучающих данных", right_count / len(reses_train))

Наивный Байессовский классификатор на обучающих данных 0.9946


In [14]:
df_test = pd.read_csv('data/news_test.txt',sep='\t', header = None)
df_test['tokens'] = df_test[2].apply(lambda x: tokenizers.simple_word_tokenize(x))
df_test['clean_words'] = df_test['tokens'].apply(lambda x: clear_words(x))


In [15]:
reses_test = []
clean_words = df_test.clean_words.values.tolist()
for i in range(len(clean_words)):
    reses_test.append([classif(clean_words[i], distr_1, distr_2), df_test[0].values.tolist()[i]])

right_count = 0
for i in reses_test:
    if (i[0] == i[1]):
        right_count += 1
print("Наивный Байессовский классификатор на тестовых данных", right_count / len(reses_test))

Наивный Байессовский классификатор на тестовых данных 0.8426666666666667


In [16]:
num_vocab = wtw.wv.vocab.copy()
for i, key in enumerate(num_vocab):
    num_vocab[key] = i
num_vocab_themes = dict()
for i, k in enumerate(themes):
    num_vocab_themes[k] = i

In [17]:
import numpy as np

def get_vector(sentence):
    res = [0] * len(num_vocab)
    for i in sentence:
        if (num_vocab.get(i,0)) != 0:
            res[num_vocab[i]]+= 1
    new_res = []
    calls = 0
    summa = 0
    for i in res:
        calls += 1
        summa += i
        if (calls == 100):
            calls = 0
            new_res.append(summa)
            summa = 0
    v = np.array(new_res)
    normalized_v = v/(np.linalg.norm(v) + 0.00000001)
    return normalized_v.tolist()

In [18]:
XXX = []
for i in df_train.clean_words.values:
    XXX.append(get_vector(i))
YYY = []
for i in df_train[0].values:
    YYY.append(num_vocab_themes[i])

In [19]:
from sklearn import svm
print(np.any(np.isnan(XXX)))
print(np.all(np.isfinite(XXX)))
svmMy = svm.SVC(kernel='rbf', gamma=5.)
svmMy.fit(XXX, YYY)

False
True


SVC(gamma=5.0)

In [20]:
XXX_test = []
for i in df_test.clean_words.values:
    XXX_test.append(get_vector(i))
YYY_test = []
for i in df_test[0].values:
    YYY_test.append(num_vocab_themes[i])

In [21]:
counter = 0
tmp = df_test[0].values
tmp2 = df_test.clean_words.values
for i in range(len(tmp)):
    if (num_vocab_themes[tmp[i]] == svmMy.predict([get_vector(tmp2[i])])[0]):
        counter += 1
print(counter / len(tmp))

0.696


In [22]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
XXX_train_rubert = []
for i in df_train[2].values:
    XXX_train_rubert.append(embed_bert_cls(i, model, tokenizer))
YYY_train_rubert = []
for i in df_train[0].values:
    YYY_train_rubert.append(num_vocab_themes[i])

In [24]:
XXX_test_rubert = []
for i in df_test[2].values:
    XXX_test_rubert.append(embed_bert_cls(i, model, tokenizer))
YYY_test_rubert = []
for i in df_test[0].values:
    YYY_test_rubert.append(num_vocab_themes[i])

In [25]:
from sklearn import svm
print(np.any(np.isnan(XXX_train_rubert)))
print(np.all(np.isfinite(XXX_train_rubert)))
print(len(XXX_train_rubert))
svmMy_rubert = svm.SVC(kernel='rbf', gamma=5.)
svmMy_rubert.fit(XXX_train_rubert, YYY_train_rubert)

False
True
15000


SVC(gamma=5.0)

In [26]:
counter = 0
for i in range(len(XXX_test_rubert)):
    if (YYY_test_rubert[i] == svmMy_rubert.predict([XXX_test_rubert[i]])[0]):
        counter += 1
print(counter / len(tmp))

0.8393333333333334
